# AWS - Financial Services Cloud Symposium - 2022

## How Nomura built a next-generation time series Metrics-as-a-Service

- Andy Coates - Executive Director, Global Head of Infrastructure Development, Nomura
- Arnav Khare - Principal Solutions Architect, Global Financial Services, AWS

### Serverless analytics on AWS
- pillars for AWS analytics 
- scalable data lakes
  - explosion of data cannot be copped up using traditional analytical tools
  - AWS provides scalability S3 and Athena
- purpose bilt for performance and cost
  - reduces overhead
- serverless and easy to use
  - build use cases rather than managing servers
- unified data access, security and governance
  - difference between data swamp and data lake
  - ability of fine-grained access
- built in machine learning
  - Sagemaker, RedShift ML and others are directly integrated into data lakes
  
#### AWS Serverless options for data analytics
- AWS Glue 
  - serverless ETL engine
  - allows users to bring code and run in fully managed AWS clusters
  - feature of crawler
    - scans data on S3 and populates data catalog
    - data catalog is integrated with Amazon Athena
- Amazon Athena
  - serverless SQL engine which can scale to massive volumes of data
- Amazon Kinesis and Amazon MSK(Managed Services for Kafka)
  - streaming services

<p align="center"><img src="./images/nomura_serverless_data_analytics.png"></p>

$\tiny{\text{AWS Symposium 2022 - Nomura}}$

#### Serverless analytics pipeline using Glue and Athena
- use Glue to connect to on-prem datasources and query using JDBC/ODBC
- inject files using Glue and transfer using AWS DataSync
- once data comes in S3 buckets, Glue can be used to cleanse, enrich and transform the data
  - then can be consumed in downstream ML or other applications

<p align="center"><img src="./images/nomura_serverless_pipeline.png"></p>

$\tiny{\text{AWS Symposium 2022 - Nomura}}$

### Vision for infrastructure platform at Nomura
#### Introduction to Nomura
#### In 2010, Vision for the platform
- measuring/checks but threw the data away
- identify hot-spots in the infrastructure and assist with root-cause analysis
- forward looking capacity planning

### Journey and lessons learned
#### Overview of platform design
- Data collection
  - gather data globally from all systems
  - metrics baseline
- ETL
  - standard format
- Processing and Analysis
  - statistical analysis
- Storage
- Query and Visualization

#### Evolving with infrastructure estate
- In 2010 
  - focussed on the KPIs of servers only (CPU, memory usage, cores of CPU, network and disk/IO statistics)
  - about 8 billion data points every day
- In 2015
  - focused on both servers and desktop states
- In 2022
  
<p align="center"><img src="./images/nomura_infrastructure_evolution.png"></p>

$\tiny{\text{AWS Symposium 2022 - Nomura}}$

#### Phase 1 - Initial launch of the platform - 2010
- load of Java batch job for ETL and processing
- SQL to query it, Spring MVC application
- KPIs were presented in charts and unified manner, which was very helpful in times of failures especially
- added value, so started making predictions

<p align="center"><img src="./images/nomura_infrastructure_phase1.png"></p>

$\tiny{\text{AWS Symposium 2022 - Nomura}}$

#### Phase 2 - Evolving to big data - 2015
- batch processes were vertically scaling
- number of regional jobs were taking more than 8 products
- open up the metric platform, open for everyone "as-a-service"
- realized opening up database resulted in poor architecture/practices around 
- moved into Hadoop
- moved processing into Scala running in Spark
- moved from mySQL to time-series OpenTSDB and querying using REST protocol
- x22 times improvement

<p align="center"><img src="./images/nomura_infrastructure_phase2.png"></p>

$\tiny{\text{AWS Symposium 2022 - Nomura}}$

- opened metric platform as __"metric-as-a-service"__

<p align="center"><img src="./images/nomura_infrastructure_metric-as-a-service.png"></p>

$\tiny{\text{AWS Symposium 2022 - Nomura}}$

### Current architecture on AWS
#### Goals for cloud transformation
- Hadoop ecosystem were cumbersome to manage
- move towards serverless architecture
- scaling servers meant to buy and maintain servers
- disc failures would lead to backlog of data in the pipelines
- opening up as metric-as-a-service made multi-tenancy at storage layer where everything was going through one pipeline
  - there was a need to break this out

#### Transforming to public cloud - 2022
- changes were at the 3 middle layers
- moved ETL to Lambda/Python
- moved processing/analytics into AWS Glue
- moved storage into AWS S3 and Athena

<p align="center"><img src="./images/nomura_infrastructure_phase3.png"></p>

$\tiny{\text{AWS Symposium 2022 - Nomura}}$

#### High level logical architecture

<p align="center"><img src="./images/nomura_infrastructure_high-level-arch.png"></p>

$\tiny{\text{AWS Symposium 2022 - Nomura}}$

#### Detailed logical pipeline view
- In the ETL pipeline
  - the first lambda processes bundles of files from data producers
    - this is processed and put onto queue
  - the second lambda unbundles these files and processes them into chunks of JSONs
    - JSONs are then put onto another queue
  - the third lambda aggregates them into Parquet files
- this triggers the Glue jobs which scales the number of workers based on how much data needs to be processed
  - aggegation is done at minute, hour and day level of aggregation -- this dataset is queried using Athena
  - Athena is not exposed directly but is done using lambda and loadbalancer in front
- infrastructure and application level of data is processed using different pipelines
  
<p align="center"><img src="./images/nomura_infrastructure_detailed-logical-pipeline-arch.png"></p>

$\tiny{\text{AWS Symposium 2022 - Nomura}}$

#### Thinking differently in cloud
- started the journey while migrating to Hadoop
  - decomposing the architecture
- moving to AWS brought in serverless capability
- kept control of how people will query the system
  - observability and monitoring how people are querying/behaving is important
  - keep control of it
  

### Future plans
- data collection has not changed
  - modernization of data collection is next in plan
- on-premise infrastructure are leveraging AWS services
  - unify the tools of on-premise and cloud data infrastructures management
- along with metrics, logs and traces are planned to manage centrally
- apply machine learning tools to apply on data stored on S3


### Q&A
- Why are there 3 lambdas in ETL 
  - looked at Kinesis as alternative and other ways of combining this
  - this seemed to be the best for us
  - can also be done through step function
  - step function works well when there is a complex chain of processes and decision chain of processes
  - this was a case of linear 
- What is the data volume, how are the metrics pulled in, what type of metrics
  - 35 billion data point on infrastructure side
    - 400,000 datapoint per second
  - 2TB of files that arrive in the pipeline
    - this translates to 4TB of JSONs 
    - this translates to 40G of parquet files 
  - the data is pulled using different mechanism
    - infrastructure data is pulled in, 
    - stored in regional hubs as part of monitoring system responsibility
    - this is then fetched in periodically
    - these buckets of data are then fetched in
    - for non-infrastructure, endpoints are provided 
      - people push in data
- is it custom metrics, cost controls as these things can blow-out
  - on-premise service and desktop metrics
    - virtualization and group information
    - cost control - indirectly
      - request each consumer can make
      - no usage of select * allowed
    - based on query pattern 
      - change what will be allowed and not
- how many dimensions do you have for a metric, does the service scale
  - limits on number of dimension
  - dont store raw data, only store min, max or other aggregated result
    - all have different retention period based on aggregation
  - max of 10 per entity
- what is SQS used for
  - SQS is used before using the Glue job
  - used to allow the flow control

## How three Fintechs are transforming Financial Services with AWS
- Chris Ciompi, Head of Marketing, Clarity AI
- Adam Healy, CSO, Blockfi
- Ali Heron, CTO, Petal
- Moderated by Kathryn Van Nuys, Global Head, Fintech Business Development, Startups and Venture Capital, AWS

## How the Options Clearing Corporation modernized its clearing technology with Ness
- Paul Meulbroek - Head of Cloud and Data, Ness Digital Engineering
- Amy McCormick - Chief Information Officer, OCC
- Dave Hoag - Managing Director, Market Risk and Default Management, OCC

## How Hyland helped Finra to securely manage their most critical applications
- Steve Comer, AVP - Financial Services and Insurance, Hyland

## Building climate resiliency and sustainability into your IT infrastructure
- Elias Ghanem, Global Head of Capgemini Research Institute for Financial Services, Capgemini
- Ravi Khokhar, Executive Vice President & Head of Cloud at Capgemini Financial Services, Capgemini

## The Goldman Sachs Cloud Fast Track: Secure Cloud Deployments at Scale
- Ryan McDonough, Vice President, Cloud Enablement, Goldman Sachs
- Shubham Shukla, Vice President, Cloud Enablement, Goldman Sachs

## Santander's journey to achieving security at-scale on AWS
- Jorge Álvarez Fernández, Global Head of Public Cloud Architecture, Santander
- Luis Enríquez, Group VP, CTO for Cloud, Infrastructure & Automation, Santander

## How USAA built a paved road with guardrails
- Gerum Haile, VP, Chief Architect and Technical Fellow, USAA

## Shift record retention from obligation to advantage with AWS
- Russell Lewis, Financial Services Compliance Specialist, AWS
- Alket Memushaj, Principal Solutions Architect, Capital Markets, AWS

## Transform cross-border payments compliance with AWS
- Gloria Vargas, Account Manager, Global Financial Services, AWS


## How Fidelity asset management’s equity trading platform migrated all-in to AWS
- John Kerin, Head of Engineering, Asset Management - Equity Technology, Fidelity
- Girish Maraliga, SVP, Head of Architecture & Platforms, Fidelity

## How Fannie Mae is modernizing its technology, people, and processes with AWS
- Arif Sattar, Senior Director, Enterprise Cloud Migration, Fannie Mae



## Mastercard’s journey from on premises to an ML-enables SaaS solution for fraud detection
- Manu Thapar, CTO, Mastercard, AI, Security and Fraud Products



## How business transformation is driving mainframe modernization in Financial Services
- Steve Steuart, Global Head of Mainframe Migration and Modernization, AWS



## Unlock business value with document processing ML pipelines
- Mojgan Ahmadi, Principal Solutions Architect - Financial Services, AWS
- David Kheyman, Senior Solutions Architect - Financial Services, AWS

## Truist digital straddle: A customer-first approach to merger integration
- Ken Meyer, EVP & CIO, Experience and Innovation, Truist



## How Prudential’s data platform drives inclusive innovation at scale
- Shihas Vamanjoor, Vice President, Enterprise Data Platforms, Prudential
- Prasanna Sridharan, Senior Data & Analytics Architect, AWS

## How Liberty Mutual empowers developers to drive Innovation
- Matt Coulter, Senior Architect, Liberty Mutual



## How The Clearing House creates frictionless experiences for their bank customers
- Elena Whisler, VP, Relationship Management and Sales, The Clearing House



## Leveraging AWS AI/ML to support the customer experience for financial institutions
- Alvin Huang, Business Development, AWS



## How Global Payments accelerates innovation on AWS
- Scott Carter, CTO, Global Payments
- Raghu Ram Bongula, SVP Engineering, Global Payments

## How JPMC modernized its hedging and risk management platform
- Paul Bauerschmidt, Managing Director, JPMorgan Chase



## How Vanguard sped up its internal approval process for using AWS services
- Michael Bakay, IT Project Manager, Vanguard Cloud Platforms, Vanguard
- Rebecca Sundheim, Head of Developer Experience, Vanguard Cloud Enablement, Vanguard

## How Munich Re built an insurance-focused IoT solution on AWS
- Lewis Wight, CTO, Meshify HSB (Munich Re Group)
